# ChemBERTa Pre-trained Molecular Embeddings

**Hypothesis**: Pre-trained molecular embeddings may generalize better to unseen solvents and potentially break the CV-LB relationship.

**Rationale**:
- The GNN benchmark achieved CV 0.0039 (5x better than our best CV 0.008194)
- The key difference is likely pre-training on large molecular datasets
- ChemBERTa was trained on 77M molecules from ZINC database
- Pre-trained embeddings capture chemical knowledge that may transfer to unseen solvents

**Implementation**:
1. Use ChemBERTa to generate 768-dim embeddings for each solvent SMILES
2. Replace Spange/DRFP/ACS features with ChemBERTa embeddings
3. Add Arrhenius kinetics features (1/T, ln(t), interaction)
4. Train MLP on ChemBERTa embeddings + kinetics

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Load ChemBERTa
from transformers import AutoTokenizer, AutoModel

model_name = 'seyonec/ChemBERTa-zinc-base-v1'
print(f'Loading {model_name}...')
chemberta_tokenizer = AutoTokenizer.from_pretrained(model_name)
chemberta_model = AutoModel.from_pretrained(model_name).to(device)
chemberta_model.eval()
print('ChemBERTa loaded successfully!')

Loading seyonec/ChemBERTa-zinc-base-v1...


ChemBERTa loaded successfully!


In [3]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [4]:
# Load SMILES lookup and generate ChemBERTa embeddings
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {SMILES_DF.shape}')

# Generate embeddings for all solvents
CHEMBERTA_EMBEDDINGS = {}

for solvent in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent, 'solvent smiles']
    
    # Handle mixture solvents (e.g., "Water.Acetonitrile")
    if '.' in smiles:
        # For mixtures, average the embeddings of components
        parts = smiles.split('.')
        embeddings = []
        for part in parts:
            inputs = chemberta_tokenizer(part, return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = chemberta_model(**inputs)
                emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS token
            embeddings.append(emb)
        # Average the embeddings
        CHEMBERTA_EMBEDDINGS[solvent] = np.mean(embeddings, axis=0).squeeze()
    else:
        inputs = chemberta_tokenizer(smiles, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = chemberta_model(**inputs)
            emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS token
        CHEMBERTA_EMBEDDINGS[solvent] = emb.squeeze()

print(f'Generated ChemBERTa embeddings for {len(CHEMBERTA_EMBEDDINGS)} solvents')
print(f'Embedding dimension: {CHEMBERTA_EMBEDDINGS["Methanol"].shape}')

SMILES lookup: (26, 1)


Generated ChemBERTa embeddings for 26 solvents
Embedding dimension: (768,)


In [5]:
# Create DataFrame for ChemBERTa embeddings
CHEMBERTA_DF = pd.DataFrame(CHEMBERTA_EMBEDDINGS).T
CHEMBERTA_DF.index.name = 'SOLVENT NAME'
print(f'ChemBERTa DataFrame: {CHEMBERTA_DF.shape}')
print(CHEMBERTA_DF.head())

ChemBERTa DataFrame: (26, 768)
                                        0         1         2         3    \
SOLVENT NAME                                                                
Cyclohexane                        2.490830  1.924160  0.295041 -2.139060   
Ethyl Acetate                      1.831127  1.058668  0.755435 -1.189752   
Acetic Acid                        0.455353  0.285686 -1.085794 -1.166156   
2-Methyltetrahydrofuran [2-MeTHF]  3.420858  0.539570  0.030612 -1.840547   
1,1,1,3,3,3-Hexafluoropropan-2-ol  1.213224  0.286342  0.117680 -0.057614   

                                        4         5         6         7    \
SOLVENT NAME                                                                
Cyclohexane                        0.529282  0.217540 -0.515373 -0.439706   
Ethyl Acetate                     -0.021727 -0.906612  0.303518 -0.691459   
Acetic Acid                       -0.172513 -1.243419 -0.003198  0.567491   
2-Methyltetrahydrofuran [2-MeTHF]  0.469968 

In [6]:
# ChemBERTa Featurizer
class ChemBERTaFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.chemberta_df = CHEMBERTA_DF
        # 5 kinetics features + 768 ChemBERTa features = 773 total
        self.feats_dim = 5 + self.chemberta_df.shape[1]

    def featurize(self, X, flip=False):
        # Kinetics features
        temp_c = X["Temperature"].values.reshape(-1, 1)
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([temp_c, time_m, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_emb = self.chemberta_df.loc[X["SOLVENT A NAME"]].values
            B_emb = self.chemberta_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_emb = B_emb * (1 - (1-pct)) + A_emb * (1-pct)
            else:
                X_emb = A_emb * (1 - pct) + B_emb * pct
        else:
            X_emb = self.chemberta_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_emb])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.float32)

print(f'ChemBERTa feature dimension: {ChemBERTaFeaturizer().feats_dim}')

ChemBERTa feature dimension: 773


In [7]:
# MLP Model for ChemBERTa embeddings
class ChemBERTaMLP(nn.Module):
    def __init__(self, input_dim=773, hidden_dim=256, output_dim=3, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

print('ChemBERTa MLP model defined')

ChemBERTa MLP model defined


In [8]:
# ChemBERTa Model Wrapper
class ChemBERTaModel:
    def __init__(self, data='single', epochs=300, lr=0.001, hidden_dim=256, dropout=0.3):
        self.data = data
        self.epochs = epochs
        self.lr = lr
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.model = None
        self.scaler = StandardScaler()
        self.featurizer = ChemBERTaFeaturizer(mixed=(data == 'full'))
        
    def train_model(self, X, Y):
        # Get features
        features = self.featurizer.featurize(X)
        features_scaled = self.scaler.fit_transform(features)
        
        # Convert to tensors
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        Y_tensor = torch.tensor(Y.values, dtype=torch.float32).to(device)
        
        # Initialize model
        self.model = ChemBERTaMLP(
            input_dim=self.featurizer.feats_dim,
            hidden_dim=self.hidden_dim,
            output_dim=3,
            dropout=self.dropout
        ).to(device)
        
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=30, factor=0.5)
        criterion = nn.MSELoss()
        
        # Training loop
        self.model.train()
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            predictions = self.model(X_tensor)
            loss = criterion(predictions, Y_tensor)
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
            
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 50:
                    break
        
        return self
    
    def predict(self, X):
        self.model.eval()
        
        # Get features
        features = self.featurizer.featurize(X)
        features_scaled = self.scaler.transform(features)
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        
        with torch.no_grad():
            predictions = self.model(X_tensor)
        
        # TTA for mixed solvents
        if self.data == 'full':
            features_flip = self.featurizer.featurize(X, flip=True)
            features_flip_scaled = self.scaler.transform(features_flip)
            X_flip_tensor = torch.tensor(features_flip_scaled, dtype=torch.float32).to(device)
            with torch.no_grad():
                predictions_flip = self.model(X_flip_tensor)
            predictions = (predictions + predictions_flip) / 2
        
        predictions = torch.clamp(predictions, 0, 1)
        return predictions

print('ChemBERTa model wrapper defined')

ChemBERTa model wrapper defined


In [9]:
# Quick test
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on a small subset
X_test = X_single.iloc[:50]
Y_test = Y_single.iloc[:50]

model = ChemBERTaModel(data='single', epochs=50)
model.train_model(X_test, Y_test)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')
print(f'Test predictions range: [{preds.min():.4f}, {preds.max():.4f}]')

Single solvent data: X=(656, 3), Y=(656, 3)
Test predictions shape: torch.Size([50, 3])
Test predictions range: [0.1344, 0.5689]


In [10]:
# Run CV on single solvent data
print('\n=== Single Solvent CV (ChemBERTa) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ChemBERTaModel(data='single', epochs=300, lr=0.001)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.cpu().numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')


=== Single Solvent CV (ChemBERTa) ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:00<00:08,  2.85it/s]

  8%|▊         | 2/24 [00:00<00:07,  2.84it/s]

 12%|█▎        | 3/24 [00:01<00:07,  2.85it/s]

...

 92%|█████████▏| 22/24 [00:07<00:00,  2.90it/s]

 96%|█████████▌| 23/24 [00:08<00:00,  2.85it/s]

100%|██████████| 24/24 [00:08<00:00,  2.86it/s]

100%|██████████| 24/24 [00:08<00:00,  2.87it/s]

Single Solvent MSE: 0.029987 (n=656)


In [11]:
# Run CV on full data
print('\n=== Full Data CV (ChemBERTa) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ChemBERTaModel(data='full', epochs=300, lr=0.001)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.cpu().numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')


=== Full Data CV (ChemBERTa) ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:00<00:04,  2.65it/s]

 15%|█▌        | 2/13 [00:00<00:04,  2.57it/s]

 23%|██▎       | 3/13 [00:01<00:03,  2.58it/s]

...

 85%|████████▍ | 11/13 [00:03<00:00,  2.87it/s]

 92%|█████████▏| 12/13 [00:04<00:00,  2.88it/s]

100%|██████████| 13/13 [00:04<00:00,  2.87it/s]

100%|██████████| 13/13 [00:04<00:00,  2.80it/s]

Full Data MSE: 0.035375 (n=1227)


In [12]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (ChemBERTa) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')
print(f'GNN Benchmark: 0.0039')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')


=== CV SCORE SUMMARY (ChemBERTa) ===
Single Solvent MSE: 0.029987 (n=656)
Full Data MSE: 0.035375 (n=1227)
Overall MSE: 0.033498

Best CV (exp_032): 0.008194
GNN Benchmark: 0.0039

✗ WORSE: 308.81% worse than best CV


# Hybrid ChemBERTa + Tabular Features\n\nThe pure ChemBERTa approach performed poorly (CV 0.033498, 308% worse than best).\nLet's try combining ChemBERTa embeddings with the domain-specific features (Spange + DRFP + ACS PCA)."

In [ ]:
# Load tabular feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

In [ ]:
# Hybrid Featurizer: ChemBERTa + Spange + DRFP + ACS PCA
class HybridChemBERTaFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.chemberta_df = CHEMBERTA_DF
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        # 5 kinetics + 768 ChemBERTa + 13 Spange + 122 DRFP + 5 ACS PCA = 913 total
        self.feats_dim = 5 + self.chemberta_df.shape[1] + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        # Kinetics features
        temp_c = X["Temperature"].values.reshape(-1, 1)
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([temp_c, time_m, inv_temp, log_time, interaction])
        
        if self.mixed:
            # ChemBERTa
            A_chemberta = self.chemberta_df.loc[X["SOLVENT A NAME"]].values
            B_chemberta = self.chemberta_df.loc[X["SOLVENT B NAME"]].values
            # Spange
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            # DRFP
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            # ACS PCA
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_chemberta = B_chemberta * (1 - (1-pct)) + A_chemberta * (1-pct)
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_chemberta = A_chemberta * (1 - pct) + B_chemberta * pct
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_chemberta = self.chemberta_df.loc[X["SOLVENT NAME"]].values
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_chemberta, X_spange, X_drfp, X_acs])

print(f'Hybrid feature dimension: {HybridChemBERTaFeaturizer().feats_dim}')

In [ ]:
# Hybrid ChemBERTa Model Wrapper
class HybridChemBERTaModel:
    def __init__(self, data='single', epochs=300, lr=0.001, hidden_dim=256, dropout=0.3):
        self.data = data
        self.epochs = epochs
        self.lr = lr
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.model = None
        self.scaler = StandardScaler()
        self.featurizer = HybridChemBERTaFeaturizer(mixed=(data == 'full'))
        
    def train_model(self, X, Y):
        # Get features
        features = self.featurizer.featurize(X)
        features_scaled = self.scaler.fit_transform(features)
        
        # Convert to tensors
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        Y_tensor = torch.tensor(Y.values, dtype=torch.float32).to(device)
        
        # Initialize model
        self.model = ChemBERTaMLP(
            input_dim=self.featurizer.feats_dim,
            hidden_dim=self.hidden_dim,
            output_dim=3,
            dropout=self.dropout
        ).to(device)
        
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=30, factor=0.5)
        criterion = nn.MSELoss()
        
        # Training loop
        self.model.train()
        best_loss = float('inf')\n        patience_counter = 0
        
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            predictions = self.model(X_tensor)
            loss = criterion(predictions, Y_tensor)
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
            
            if loss.item() < best_loss:
                best_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 50:
                    break
        
        return self
    
    def predict(self, X):
        self.model.eval()
        
        # Get features
        features = self.featurizer.featurize(X)
        features_scaled = self.scaler.transform(features)
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
        
        with torch.no_grad():
            predictions = self.model(X_tensor)
        
        # TTA for mixed solvents
        if self.data == 'full':
            features_flip = self.featurizer.featurize(X, flip=True)
            features_flip_scaled = self.scaler.transform(features_flip)
            X_flip_tensor = torch.tensor(features_flip_scaled, dtype=torch.float32).to(device)
            with torch.no_grad():
                predictions_flip = self.model(X_flip_tensor)
            predictions = (predictions + predictions_flip) / 2
        
        predictions = torch.clamp(predictions, 0, 1)
        return predictions

print('Hybrid ChemBERTa model wrapper defined')

In [ ]:
# Run CV on single solvent data with hybrid model
print('\\n=== Single Solvent CV (Hybrid ChemBERTa) ===')\nX_single, Y_single = load_data(\"single_solvent\")\n\nsplit_generator = generate_leave_one_out_splits(X_single, Y_single)\nall_predictions_single_hybrid = []\nall_actuals_single_hybrid = []\n\nfor fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):\n    (train_X, train_Y), (test_X, test_Y) = split\n    \n    model = HybridChemBERTaModel(data='single', epochs=300, lr=0.001)\n    model.train_model(train_X, train_Y)\n    predictions = model.predict(test_X)\n    \n    all_predictions_single_hybrid.append(predictions.cpu().numpy())\n    all_actuals_single_hybrid.append(test_Y.values)\n\npreds_single_hybrid = np.vstack(all_predictions_single_hybrid)\nactuals_single_hybrid = np.vstack(all_actuals_single_hybrid)\nmse_single_hybrid = np.mean((preds_single_hybrid - actuals_single_hybrid) ** 2)\nprint(f'Single Solvent MSE (Hybrid): {mse_single_hybrid:.6f} (n={len(preds_single_hybrid)})')

In [ ]:
# Run CV on full data with hybrid model
print('\\n=== Full Data CV (Hybrid ChemBERTa) ===')\nX_full, Y_full = load_data(\"full\")\n\nsplit_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)\nall_predictions_full_hybrid = []\nall_actuals_full_hybrid = []\n\nfor fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):\n    (train_X, train_Y), (test_X, test_Y) = split\n    \n    model = HybridChemBERTaModel(data='full', epochs=300, lr=0.001)\n    model.train_model(train_X, train_Y)\n    predictions = model.predict(test_X)\n    \n    all_predictions_full_hybrid.append(predictions.cpu().numpy())\n    all_actuals_full_hybrid.append(test_Y.values)\n\npreds_full_hybrid = np.vstack(all_predictions_full_hybrid)\nactuals_full_hybrid = np.vstack(all_actuals_full_hybrid)\nmse_full_hybrid = np.mean((preds_full_hybrid - actuals_full_hybrid) ** 2)\nprint(f'Full Data MSE (Hybrid): {mse_full_hybrid:.6f} (n={len(preds_full_hybrid)})')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ChemBERTaModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ChemBERTaModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Final verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')
print(f'GNN Benchmark: 0.0039')